loading the libraries

In [ ]:
#importing the libraries
#  Visualization
library(ggplot2)

# Data manipulation
library(tibble)
library(dplyr)
library(tidyr)
library(purrr)
library(readr)

# Modelling and Preprocessing
library(tidymodels)

# Data splitting and resampling
library(rsample) # train_test_split_equivalent
library(tune) # GridSearchCV equivalent
library(workflows) #make_pipeline equivalent
library(recipes) #make_column_transformer and StandardScaler equivalent

# Metrics
library(yardstick) # for mean squared error

#Time and Dates
library(lubridate)


#formatting graphs
options(repr.plot.width=12, repr.plot.height=6)

Step 2: Loading data in R

In [ ]:
#loading data into R
players<-read_csv("players.csv")
players
sessions<- read_csv("sessions.csv")
sessions

In [ ]:
# CONVERT START_TIME AND END_TIME TO DATETIME
sessions<-sessions|>
mutate( start_time= dmy_hm(start_time), end_time= dmy_hm(end_time))
sessions

In [ ]:
#CONVERT THE TIMEZONE FROM GMT TO PST
sessions <- sessions |>
  mutate(
    start_time = with_tz(force_tz(start_time, tzone = "GMT"), tzone = "America/Los_Angeles"),
    end_time = with_tz(force_tz(end_time, tzone = "GMT"), tzone = "America/Los_Angeles")
  )
sessions 


WRANGLING DATA

In [ ]:
#Creating an hourly time range
hours<- seq(from= floor_date(min(sessions$start_time, na.rm = TRUE), unit= "hour"), 
            to = ceiling_date(max(sessions$end_time, na.rm = TRUE), unit= "hour"), 
            by="1 hour")
head(hours)

In [ ]:
#Count how many sessions are active during each hour
concurrent_sessions<-map_dfr(hours, function(h){
active_players <- sum(sessions$start_time <= h & sessions$end_time >= h)
tibble( time=h, active_sessions = active_players)})
head(concurrent_sessions)